# Lab CMBLenses

In [1]:
using Revise, CMBLensing, CMBLensingInferenceTestProblem, CUDA, JLD2, LaTeXStrings, LinearAlgebra, 
    MCMCChains, MCMCDiagnosticTools, MicroCanonicalHMC, MuseInference, MuseInference, Plots, 
    ProgressMeter, Random, Statistics, Zygote

LoadError: ArgumentError: Package CMBLensing [b60c06c0-7e54-11e8-3788-4bd722d65317] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [2]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

LoadError: UndefVarError: `Plots` not defined

In [3]:
Nside = 64
T = Float64

Float64

# MCHMC

## Init

In [4]:
prob = load_cmb_lensing_problem(;storage=Array, T, Nside);

LoadError: UndefVarError: `load_cmb_lensing_problem` not defined

In [5]:
target = CMBLensingTarget(prob);

LoadError: UndefVarError: `CMBLensingTarget` not defined

In [6]:
d = length(prob.Ωstart)
to_vec, from_vec = CMBLensingInferenceTestProblem.to_from_vec(prob.Ωstart);

LoadError: UndefVarError: `prob` not defined

## Sample

In [7]:
spl = MCHMC(500, 0.0001; adaptive=true, init_eps=10.0, init_L=10.0*sqrt(50), sigma=one(LenseBasis(diag(target.Λmass))));

LoadError: UndefVarError: `target` not defined

In [8]:
prob.ncalls[] = 0
samples_mchmc = Sample(spl, target, 1000, dialog=false, progress=true);
ncalls_mchmc = prob.ncalls[]

LoadError: UndefVarError: `prob` not defined

In [9]:
chain_mchmc = Chains(
    permutedims(reduce(hcat, [[exp.(sample[end÷2-2:end÷2-1]); sample[end-3:end]] for sample in samples_mchmc if all(isfinite.(sample))])),
    [:r, :Aϕ, :p_logr, :p_logAϕ, :dE, :logpdf],
);

LoadError: UndefVarError: `samples_mchmc` not defined

In [10]:
ess_mchmc = MCMCDiagnosticTools.ess(chain_mchmc)[[:r,:Aϕ],:ess]

LoadError: UndefVarError: `MCMCDiagnosticTools` not defined

In [11]:
ess_per_call_mchmc = ess_mchmc / ncalls_mchmc

LoadError: UndefVarError: `ess_mchmc` not defined

## Plot

In [12]:
plot(chain_mchmc[:r], label=L"r", xlabel="step")
plot!(chain_mchmc[:Aϕ], label=L"A_\phi")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[12]:1

In [13]:
plot(chain_mchmc[:p_logAϕ], label=L"\Pi(\log \, A_\phi)")
plot!(chain_mchmc[:p_logr], label=L"\Pi(\log \, r)", xlabel="step")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[13]:1

In [14]:
std(chain_mchmc[:dE])^2/d

LoadError: UndefVarError: `chain_mchmc` not defined

In [15]:
plot(chain_mchmc[:dE]/d, label=L"\mathrm{Energy}/d", xlabel="step")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[15]:1

In [16]:
plot(
    histogram(chain_mchmc[:r], xlabel=L"r", label=nothing, lw=1),
    histogram(chain_mchmc[:Aϕ], xlabel=L"A_\phi", label=nothing, lw=1),
    size = (700,300)
)

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[16]:2

### Animation

In [17]:
to_vec, from_vec = CMBLensingInferenceTestProblem.to_from_vec(prob.Ωstart)
last_sample = from_vec(samples_mchmc[end][1:end-2]);

LoadError: UndefVarError: `CMBLensingInferenceTestProblem` not defined

In [18]:
ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(last_sample[k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[18]:1

In [19]:
ϕs = [∇²*from_vec(sample[1:end-2]).ϕ°/2 for sample in samples_mchmc[end-500:end]]
anim = @animate for i in 1:length(ϕs)
    motionblur = [0.5, 1, 0.5]
    frame = sum(circshift(ϕs, i)[1:length(motionblur)] .* motionblur) / sum(motionblur)
    plot(1 * frame, clims=(-0.5, 0.5), c=:thermal, title="κ samples")
end;

LoadError: UndefVarError: `samples_mchmc` not defined

In [20]:
mp4(anim, "kappa_samples.mp4", fps = 25)

LoadError: UndefVarError: `mp4` not defined

# HMC

In [21]:
prob = load_cmb_lensing_problem(;storage=CuArray, T, Nside);

LoadError: UndefVarError: `CuArray` not defined

In [22]:
Ω = prob.Ωstart
samples_hmc = []
rng = Xoshiro(1)
prob.ncalls[] = 0
@showprogress for i=1:1000
    Ω, = state = hmc_step(rng, prob, Ω, prob.Λmass; symp_kwargs=[(N=25, ϵ=0.05)], progress=false, always_accept=(i<10))
    push!(samples_hmc, state)
end
ncalls_hmc = prob.ncalls[]

LoadError: UndefVarError: `prob` not defined

In [23]:
chain_hmc = Chains(
    permutedims(reduce(hcat, [exp.(sample[1].θ) for sample in samples_hmc])),
    [:r, :Aϕ],
);

LoadError: UndefVarError: `samples_hmc` not defined

## Plot

In [24]:
plot(chain_hmc[:r], label=L"r", xlabel="step")
plot!(chain_hmc[:Aϕ], label=L"A_\phi")

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[24]:1

In [25]:
ess_hmc = MCMCDiagnosticTools.ess(chain_hmc)[[:r,:Aϕ],:ess]

LoadError: UndefVarError: `MCMCDiagnosticTools` not defined

In [26]:
ess_per_call_hmc = ess_hmc ./ ncalls_hmc

LoadError: UndefVarError: `ess_hmc` not defined

# MUSE

In [27]:
using CMBLensing.ComponentArrays, MuseInference.FiniteDifferences

LoadError: ArgumentError: Package CMBLensing [b60c06c0-7e54-11e8-3788-4bd722d65317] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [28]:
prob = load_cmb_lensing_problem(;storage=CuArray, T, Nside);

LoadError: UndefVarError: `CuArray` not defined

In [29]:
muse_prob = CMBLensingMuseProblem(
    prob.ds, 
    MAP_joint_kwargs = (minsteps=3, nsteps=15, αtol=1e-2, gradtol=3e-5, progress=false, history_keys=(:logpdf, :ΔΩ°_norm)),
);

LoadError: UndefVarError: `CMBLensingMuseProblem` not defined

In [30]:
# small hack to allow getting MUSE covariance in terms of transformed θ
CMBLensingMuseInferenceExt = Base.get_extension(CMBLensing,:CMBLensingMuseInferenceExt)
CMBLensingMuseInferenceExt.mergeθ(prob::CMBLensingMuseProblem, θ) = exp.(θ)

LoadError: UndefVarError: `CMBLensing` not defined

In [31]:
# z₀ = zero(FieldTuple(MuseInference.select(NamedTuple(prob.Ωstart), (:f°, :ϕ°))))
# H_pre = Diagonal(FieldTuple(MuseInference.select(NamedTuple(prob.Λmass.diag), (:f°, :ϕ°))))
# H_pre_map = let H_pre=H_pre, z₀=z₀
#     MuseInference.LinearMap{eltype(z₀)}(length(z₀), issymmetric=true) do z
#         f, = promote(z, z₀)
#         LenseBasis(H_pre \ f)[:]
#     end
# end
# implicit_diff_cg_kwargs = (maxiter=1500,Pl=MuseInference.InverseMap(H_pre_map));

In [32]:
z₀ = zero(MuseInference.sample_x_z(muse_prob, Xoshiro(0), prob.Ωstart.θ).z);
result = MuseResult()
nsims = 200
rng = Xoshiro(0)

prob.ncalls[] = 0
MuseInference.muse!(result,  muse_prob, prob.Ωstart.θ; nsims, rng, z₀, maxsteps=2, θ_rtol=0, progress=true, save_MAPs=false)
MuseInference.get_J!(result, muse_prob; nsims,   rng, z₀, progress=true)
MuseInference.get_H!(result, muse_prob; nsims=4, rng, z₀, progress=true, step=std(result.gs)/100, fdm=central_fdm(2,1,adapt=0))
ncalls_muse = prob.ncalls[];

LoadError: UndefVarError: `MuseInference` not defined

In [33]:
chain_muse = Chains(permutedims(rand(result.dist,100000)), [:logr, :logAϕ]);

LoadError: UndefVarError: `result` not defined

## Plot

In [34]:
plot(
    map(result.history[1:1:end]) do h
        p = plot()
        for x in h.ẑ_history_sims
            plot!(first.(x), yscale=:log10)
        end
        plot!(first.(h.ẑ_history_dat), yscale=:log10, c=:black)
        p
    end...,
    size = (1000,500),
    xticks = 0:2:15,
    # ylim = (1e-7, 1e-3)
)

LoadError: UndefVarError: `result` not defined

In [35]:
plot(
    map(result.history[1:1:end]) do h
        p = plot()
        for x in h.ẑ_history_sims
            plot!(last.(x), yscale=:log10)
        end
        plot!(last.(h.ẑ_history_dat), yscale=:log10, c=:black)
        p
    end...,
    size = (1000,500),
    xticks = 0:2:15,
    ylim = (1e-7, 1e-3)
)

LoadError: UndefVarError: `result` not defined

In [36]:
plot(
    plot(getindex.([getindex.(result.history, :θ); [result.θ]], :r), m="."),
    plot(getindex.([getindex.(result.history, :θ); [result.θ]], :Aϕ), m=".")
)

LoadError: UndefVarError: `plot` not defined

In [37]:
ess_per_call_muse = nsims / ncalls_muse

LoadError: UndefVarError: `nsims` not defined

# Compare

In [38]:
which_ess = minimum # can be: first (r), last (Aphi), or minimum
bar(
    ["HMC" "MCHMC" "MUSE"],
    which_ess.([[ess_per_call_hmc] [ess_per_call_mchmc] [ess_per_call_muse]]),
    ylabel = "eff. samples / ∇logP eval"
)

LoadError: UndefVarError: `bar` not defined

In [39]:
p = plot(label=L"r", xlabel="step (scaled consistently for both)")
for k in [:r, :Aϕ]
    plot!(chain_hmc[k][1:length(chain_mchmc)÷25], c=1, label=(k==:r ? "HMC" : nothing))
    plot!(chain_mchmc[k][1:25:end], c=2, label=(k==:r ? "MCHMC" : nothing))
end
p

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[39]:1

In [40]:
plot(
    begin
        histogram(log.(chain_hmc[:r]), normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40))
        histogram!(log.(chain_mchmc[:r]), normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40))
        histogram!(chain_muse[:logr], normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40))
        vline!([prob.Ωtrue.θ.r], c=4, lw=3)
    end, 
    begin
        histogram(log.(chain_hmc[:Aϕ]), normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40))
        histogram!(log.(chain_mchmc[:Aϕ]), normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40))
        histogram!(chain_muse[:logAϕ], normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40))
        vline!([prob.Ωtrue.θ.Aϕ], c=4, lw=3)
    end, 
    size = (700, 300)
)

LoadError: UndefVarError: `histogram` not defined